<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/201_Beautiful_Soup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Beautiful Soup
==============

Web スクレイピング
------------------

**スクレイピング**（scraping）とは、「こすり落とす」という意味を持つ scrape に由来する用語で、Web やデータベースを広く探って特定の情報を抽出する手法を指す。

Web API が提供されていないサイトからデータを取得するために、Web ページのスクレイピングが行われる。ただし、以下の事項に注意する。

  * **Web サイトが利用規約においてスクレイピングによるデータ取得を禁止している場合**
  * **ダウンロードの間隔を1秒以上空けていないなどの理由で、Web サイトのサーバーに過重な負担をかける場合**
  * **取得したデータの使用が、個人や家族間での活用、情報解析、Web 検索サービスの活用といった著作権法上許容されている限度を超える場合**（情報を頒布・販売する行為など）
  * **Web サイト自体がコンテンツを不法に複製・頒布している場合**

これらの場合には、スクレイピング行為が違法とされ法的責任を問われることになる。

なお、Web 検索サービスの活用を目的とする場合は、サイトの `robots.txt` ファイルで特定のユーザーエージェントでのアクセスが許可されていない URL パスを対象としたスクレイピングをしないように気を付けること。

Web スクレイピングは、大まかに 3 つのステップに分けることができる。

  1. データの取得
  2. データの抽出
  3. データの保存

Requests パッケージは、データの取得に使用される。

一方、[Beautiful Soup](https://pypi.org/project/beautifulsoup4/) は HTML や XML を解析する機能を提供するサードパティ製パッケージであり、Web スクレイピングにおけるデータの抽出に使用される。ライセンスは MIT License。 インストール方法は次のとおり。

``` python
pip install beautifulsoup4
```

BeautifulSoup オブジェクト
--------------------------

`bs4.BeautifulSoup` オブジェクトは、与えられたマークアップが解析されて作られた木構造を表現する。コンストラクタのオプションは次のとおり。

``` python
bs4.BeautifulSoup(markup="", features=None, builder=None, parse_only=None, from_encoding=None, exclude_encodings=None, element_classes=None, **kwargs)
```

主に使用するのは、次の 3 つのオプションである。

| オプション | 意味 |
|:---|:---|
| `markup` | パースするマークアップを文字列、バイト列またはファイルオブジェクトで指定する。Requests を使って取得したデータをパースする場合は、その `text` 属性よりも `content` <br />属性を渡したほうが文字化けしない |
| `features` | 使用するパーサーを指定する。`None` を指定した場合（デフォルト）、標準ライブラリの `'html.parser'` モジュールが使用される |
| `from_encoding` | マークアップの文字コードを文字列で指定する。`None` を指定した場合（デフォルト）、文字コードが自動で検出される。自動検出の結果が間違っている場合は、文字化けとな<br />り、正しいパースが行われない |
| `exclude_encodings` | 除外する文字コードを文字列のリストで指定する。間違った文字コードが自動検出される場合は、それを含むリストを渡すと、正しい文字コードが推測されることがある |

`features` オプションに指定できるパーサーは次のとおり。

| パーサー | マークアップ | 特徴 |
|:---|:---|:---|
| `'html.parser'` | HTML | Python 標準ライブラリの `html.parser` モジュールが提供するパーサー。低速 |
| `'html5lib'` | HTML | [html5lib](https://pypi.org/project/html5lib/) パッケージが提供するパーサー。標準規格に従っていない HTML でもパースしてくれる |
| `'lxml'` | HTML | [lxml](https://pypi.org/project/lxml/) パッケージが提供するパーサー。C で書かれているため高速だが、標準規格に従っていない HTML の解析に失敗する |
| `'lxml-xml'` または `'xml'` | XML | `lxml` パッケージが提供するパーサー。XML をパースする |

基本的に `'lxml'` を指定し、うまくいかない場合は `'html.parser'` を使うようにすればよい。ただし、lxml パッケージをインストールする必要がある。

マークアップを文字列で指定する例:

In [ ]:
from bs4 import BeautifulSoup
print(BeautifulSoup("<html><head></head><body>Sacr&eacute; bleu!</body></html>"))

<html><head></head><body>Sacré bleu!</body></html>


マークアップをファイルオブジェクトで指定する例:

``` python
from bs4 import BeautifulSoup

with open("index.html") as fp:
    soup = BeautifulSoup(fp, 'lxml')
```

タグオブジェクト
----------------

`bs4.Tag` オブジェクト（以下、タグオブジェクト）は、マークアップの要素を表す。マークアップの解析により、タグオブジェクトと `bs4.NavigableString` オブジェクトを節点とするツリーが構築される。`bs4.BeautifulSoup` は `bs4.Tag` のサブクラスであり、ツリーのルートを表す。`bs4.NavigableString` は、タグで囲まれたテキストを表す。親要素のタグオブジェクトから、ドット `.` 記法と要素名で子要素のタグオブジェクトにアクセスできる（直接の子要素でなくてもよい）。ただしこの方法では、マークアップ中に同じタグが複数含まれていても、親要素のタグから見て最初に現れるものに対応するタグオブジェクトにしかアクセスできないことに注意する。

### タグオブジェクトの属性 ###

タグオブジェクトの主な属性は次のとおり。

| 属性 | 意味 |
|:---|:---|
| `name` | タグ名の文字列 |
| `attrs` | タグの属性の名前と値の組からなる辞書。複数の値を持つことが許される属性（`class` など）の値はリストになっている |
| `text` | （読み出し専用）要素のテキスト（文字列）。`get_text()` と同じ |
| `string` | 直接の子要素の `bs4.NavigableString` オブジェクト。要素がタグのある子要素を持つときは `None` である |
| `stripped_strings` | （読み出し専用）子孫要素の `bs4.NavigableString` オブジェクトを返すジェネレーター |
| `contents` | 子要素のリスト |

タグオブジェクトに対し添字表記（`[key]`）を使うと、タグの属性を参照できる。キー `key` が存在しない場合はエラーが発生する。

In [ ]:
from bs4 import BeautifulSoup
import requests

response = requests.get("https://www.python.org/")
soup = BeautifulSoup(response.content, "lxml")
print(f"{soup.title=}")
print(f"{soup.title.name=}")
print(f"{soup.title.text=}")
print(f"{soup.h1=}")
print(f"{soup.h1.contents=}")
print(f"{soup.img=}")
print(f"{soup.img.attrs=}")
print(f"{soup.img['src']=}")
try:
    print(f"{soup.img['hoge']=}")
except KeyError:
    print("'hoge'キーは存在しない")

soup.title=<title>Welcome to Python.org</title>
soup.title.name='title'
soup.title.text='Welcome to Python.org'
soup.h1=<h1 class="site-headline">
<a href="/"><img alt="python™" class="python-logo" src="/static/img/python-logo.png"/></a>
</h1>
soup.h1.contents=['\n', <a href="/"><img alt="python™" class="python-logo" src="/static/img/python-logo.png"/></a>, '\n']
soup.img=<img alt="python™" class="python-logo" src="/static/img/python-logo.png"/>
soup.img.attrs={'class': ['python-logo'], 'src': '/static/img/python-logo.png', 'alt': 'python™'}
soup.img['src']='/static/img/python-logo.png'
'hoge'キーは存在しない


`text` 属性は、要素内のタグで囲まれた文字列（テキスト）を再帰的に取得し、連結する。子要素のテキストも含まれた文字列を取得できる。改行も含まれることに注意する。

一方、`string` 属性は、直接の子要素の `bs4.NavigableString` オブジェクトを得る。タグに囲まれたテキストもツリーの節点となっていて、`bs4.NavigableString` オブジェクトがこれに対応している。`string` 属性は、タグオブジェクトから延びる枝が `bs4.NavigableString` オブジェクトだけである場合に、それを参照し、そうでない場合は `None` を返すプロパティである。

タグの中に複数の要素がある場合でも、`stripped_strings` 属性で文字列だけを確認することができる。`stripped_strings` は余分な空白（改行を含む）を削除した文字列を返す。

In [ ]:
from bs4 import BeautifulSoup, NavigableString

html_doc = '''<body>
<div>
<p class="overview">テスト <b>TEST</b>てすと</p>
</div>
</body>
'''
soup = BeautifulSoup(html_doc, "html.parser")
print(f"{soup.text=}")
print(f"{soup.div.text=}")
print(f"{soup.p.text=}")
print(f"{soup.b.text=}")
print(f"{soup.string=}")
print(f"{soup.div.string=}")
print(f"{soup.p.string=}")
print(f"{soup.b.string=}")
assert isinstance(soup.b.string, NavigableString)  # string 属性が返すのは文字列ではない
print([x for x in soup.stripped_strings])

soup.text='\n\nテスト TESTてすと\n\n\n'
soup.div.text='\nテスト TESTてすと\n'
soup.p.text='テスト TESTてすと'
soup.b.text='TEST'
soup.string=None
soup.div.string=None
soup.p.string=None
soup.b.string='TEST'
['テスト', 'TEST', 'てすと']


### タグオブジェクトのメソッド ###

``` python
tag.prettify(encoding=None, formatter="minimal")
```

このメソッドは、ツリーをタグ付きのテキストとして書式化して返す。

In [ ]:
from bs4 import BeautifulSoup

html_doc = '''<html><head><title>テスト HTML</title></head>
<body>
<div>
<p class="overview">テスト<b>TEST</b>てすと
'''
soup = BeautifulSoup(html_doc, "html.parser")
print(soup.prettify())

<html>
 <head>
  <title>
   テスト HTML
  </title>
 </head>
 <body>
  <div>
   <p class="overview">
    テスト
    <b>
     TEST
    </b>
    てすと
   </p>
  </div>
 </body>
</html>



``` python
tag.get(key, default=None)
```

このメソッドは、名前が `key` の属性の値を返す。そのような属性が存在しない場合は、`default` を返す。`tag.attrs.get(key, default)` と等価である。

In [ ]:
from bs4 import BeautifulSoup

html_doc = '<div><p class="overview">テスト</p></div>'
soup = BeautifulSoup(html_doc, "html.parser")
assert soup.p.get("class") ==  ["overview"]

``` python
tag.find(name=None, attrs={}, recursive=True, string=None, **kwargs)
```

このメソッドは、引数に指定した検索条件にマッチした最初の要素（タグオブジェクトか `bs4.NavigableString` オブジェクト）を返す。`tag.p` は `tag.find("p")` と等価である。

| 引数 | 意味 |
|:---|:---|
| `name` | 要素の名前（タグ名）で検索する |
| `attrs` | 辞書で指定した要素の属性名と値で検索する |
| `recursive` | `False` の場合、直下の子要素のみが検索対象となる |
| `string` | タグで囲まれたテキストを指定する。これを指定すると、戻り値の型は `bs4.NavigableString` オブジェクトとなる |
| `kwargs` | キーワード引数として属性の名前と値を与えてフィルターを指定する。ただし、`class` は Python の予約語なのでアンダースコアを付けて `class_` とする |

`string` や `kwargs` でのマッチは完全一致である。値に正規表現オブジェクトを指定して、正規表現でフィルタリングすることもできる。

複数の引数で検索条件を指定した場合は、それらの条件の AND 検索が行われる。

In [ ]:
from bs4 import BeautifulSoup, NavigableString
import re

html_doc = """<div>
<p>属性なし</p>
<p class="overview">class 属性あり</p>
<p class="overview" id="123">class 属性と id 属性あり</p>
<p><a href="https://www.python.org/">python.org トップページ</a></p>
<p><a href="https://docs.python.org/ja/3/">Python 公式ドキュメント</a></p>
</div>
"""
soup = BeautifulSoup(html_doc, "html.parser")
assert soup.find("p").text == "属性なし"  # soup.p.text と等価
assert soup.find(attrs={"class": "overview"}).text == "class 属性あり"
assert soup.find("p", class_="overview", id="123").text == "class 属性と id 属性あり"
assert soup.find(href=re.compile(r"/3/$")).text == "Python 公式ドキュメント"
t = soup.find(string="python.org トップページ")
assert isinstance(t, NavigableString)  # 戻り値の型は bs4.NavigableString
print(t)

python.org トップページ


``` python
tag.find_all(name=None, attrs={}, recursive=True, string=None, limit=None, **kwargs)
```

このメソッドは、引数に指定した検索条件にマッチした要素（タグオブジェクトか `bs4.NavigableString` オブジェクト）のリスト `bs4.ResultSet` オブジェクトを返す（`bs4.ResultSet` は `list` のサブクラスである）。引数の `limit` に指定した数だけ要素が見つかった時点で探索を終了する。`limit` が `None` の場合（デフォルト）、すべての要素を探索する。その他の引数の意味は、`find()` メソッドと同様である。

タグオブジェクトは呼び出し可能オブジェクトでもあり、`()` を付けて呼び出すことができる。`tag(*args, **kwargs)` は `tag.find_all(*args, **kwargs)` と等価である。

In [ ]:
for elem in soup.find_all("p", class_="overview"):
    print(elem)
assert soup("a") == soup.find_all("a")

<p class="overview">class 属性あり</p>
<p class="overview" id="123">class 属性と id 属性あり</p>


``` python
tag.select(selector, namespaces=None, limit=None, **kwargs)
```

このメソッドは、CSS セレクターを使って、マッチするタグオブジェクトのリスト `bs4.ResultSet` オブジェクトを返す。

| 引数 | 意味 |
|:---|:---|
| `selector` | CSS セレクターを文字列で指定する |
| `namespaces` | XML の名前空間を辞書で指定する |
| `limit` | 指定した数だけ要素が見つかった時点で探索を終了する |

In [ ]:
print(f'{soup.select("p > a")=}')
print(f'{soup.select("p.overview")=}')

soup.select("p > a")=[<a href="https://www.python.org/">python.org トップページ</a>, <a href="https://docs.python.org/ja/3/">Python 公式ドキュメント</a>]
soup.select("p.overview")=[<p class="overview">class 属性あり</p>, <p class="overview" id="123">class 属性と id 属性あり</p>]


タグオブジェクトの属性は、一部の読み出し専用プロパティを除いて、変更可能である。この変更は、直ちにツリー全体に反映される。なお、テキストを上書きするときは、`text` 属性ではなく、`string` 属性を使わなければならない（`text` 属性は読み出し専用プロパティである）。たとえば、次のコードは、`<div>` タグを `<p>` タグに変更し、テキストを大文字に変換している。

In [ ]:
from bs4 import BeautifulSoup

html_doc = """<body>
<div>spam</div>
<div>ham</div>
<div>eggs</div>
</body>
"""
soup = BeautifulSoup(html_doc, "html.parser")
for elem in soup("div"):
    elem.name = "p"
    elem.string = elem.text.upper()
print(soup)

<body>
<p>SPAM</p>
<p>HAM</p>
<p>EGGS</p>
</body>



``` python
tag.select_one(selector, namespaces=None, **kwargs)
```

このメソッドは、CSS セレクターを使って、マッチする最初のタグオブジェクトを返す。引数の意味は `select()` メソッドと同じ。

``` python
tag.decompose()
```

このタグを削除する。子要素もすべて削除される。

``` python
tag.clear(decompose=False)
```

子要素をすべて削除する。`decompose` 引数を `True` に指定すると、子要素の削除に `decompose()` メソッドが使われる。この場合、子要素に `bs4.NavigableString` オブジェクトが含まれるとエラーとなる。

なお、要素の属性を削除するためには、del 文を使って `del tag["name"]` のように書く。

クラス階層
----------

Beautiful Soup の主要なクラスの階層は次のようになっている。

``` text
PageElement
 ├───── Tag
 │            └── BeautifulSoup
 │
 │
 └──┐
       ├── NavigableString
str  ─┘      ├─ PreformattedString
               │    ├─ CData
               │    ├─ ProcessingInstruction
               │    ├─ XMLProcessingInstruction
               │    ├─ Comment
               │    ├─ Declaration
               │    ├─ Doctype
               ├─ Stylesheet
               ├─ Script
               ├─ TemplateString
               ├─ RubyTextString
               └─ RubyParenthesisString
```

以降で取り上げるメソッドは、ほとんどが `bs4.PageElement` のメソッドを継承したものとなる。したがって、`bs4.NavigableString` オブジェクトでも使うことができる。

テキストの取得
--------------

`text` 属性は、`get_text()` メソッドを引数なしで呼び出すプロパティである。`get_text()` メソッドを直接呼び出す場合、以下の引数を指定できる。

``` python
get_text(separator="", strip=False, types=object())
```

| 引数 | 意味 |
|:---|:---|
| `separator` | タグで区切られていた位置に指定した文字列を挿入する |
| `strip` | `True` を指定するとタグの周りの空白（改行を含む）を取り除く |
| `types` | `bs4.NavigableString` のサブクラスをタプルで指定できる。標準では取得できない要素を取得する際に指定する |

`get_text()` メソッドは、デフォルトでは `bs4.NavigableString` と `bs4.CData` オブジェクトを再帰的に取得し、文字列化して返す。つまり、`get_text()` と `get_text(types=(bs4.NavigableString, bs4.CData))` は等価である。

`bs4.NavigableString` のサブクラスは、次のものが定義されている。

| サブクラス | 意味 | プレフィックス | サフィックス |
|:---|:---|:---|:---|
| `bs4.element.PreformattedString` | 基底クラス。プレフィックスとサフィックスを持つ | `''` | `''` |
| `bs4.element.CData` ★ | CDATA ブロックを表すクラス | `'<![CDATA['` | `']]>'` |
| `bs4.element.ProcessingInstruction` ★ | SGML を表すクラス。SGML は XML の前身になったフォーマット | `'<?'` | `'>'` |
| `bs4.element.XMLProcessingInstruction` | XML を表すクラス | `'<?'` | `'?>'` |
| `bs4.element.Comment` ★ | HTML と XML のコメントを表すクラス | `'<!--'` | `'-->'` |
| `bs4.element.Declaration` ★ | XML の宣言を表すクラス | `'<?'` | `'?>'` |
| `bs4.element.Doctype` ★ | DOCTYPE を表すクラス | `'<!DOCTYPE '` | `'>\n'` |
| `bs4.element.Stylesheet` ★ | スタイルシートを表すクラス | | |
| `bs4.element.Script` ★ | スクリプトを表すクラス。JavaScript など | | |
| `bs4.element.TemplateString` ★ | 大きな文書の中に埋め込まれた HTML テンプレートの中で見られる文字列を表すクラス | | |
| `bs4.element.RubyTextString` | ルビ文字を指定する `<rt>` タグのテキストを表すクラス | | |
| `bs4.element.RubyParenthesisString` | ルビ代替表示文字を指定する `<rp>` タグのテキストを表すクラス | | |

★印は、名前空間 `bs4` でもインポートされている。例えば、`bs4.element.Comment` は `bs4.Comment` で参照できる。

In [ ]:
from bs4 import BeautifulSoup

html_doc = """<div>
  red
  <p>b l u e</p>
  <span> green </span>
</div>
"""

soup = BeautifulSoup(html_doc, "html.parser")
print(soup.get_text())  # soup.text と同じ
print("-" * 30)
print(soup.get_text(strip=True))
print("-" * 30)
print(soup.get_text(strip=True, separator="++"))


  red
  b l u e
 green 


------------------------------
redb l u egreen
------------------------------
red++b l u e++green


In [ ]:
from bs4 import BeautifulSoup, Script

html_doc = "<div><script>console.log(1)</script></div>"
soup = BeautifulSoup(html_doc, "html.parser")
print(soup.get_text())  # デフォルトでは <script> タグの中身は取得できない
print("-" * 30)
print(soup.get_text(types=(Script,)))


------------------------------
console.log(1)


要素の取得
----------

``` python
find_parent(name=None, attrs={}, **kwargs)
find_parents(name=None, attrs={}, limit=None, **kwargs)
```

このメソッドは、引数に指定した検索条件にマッチした親要素（タグオブジェクトか bs4.NavigableString オブジェクト）を返す。`find_parents()` メソッドはすべての親要素のリスト `bs4.ResultSet` オブジェクトを返す。引数の意味は、`find()` および `find_all()` と同様である。

``` python
find_previous_sibling(name=None, attrs={}, string=None, **kwargs)
find_previous_siblings(name=None, attrs={}, string=None, limit=None, **kwargs)
```

このメソッドは、引数に指定した検索条件にマッチした前の兄弟要素（タグオブジェクトか bs4.NavigableString オブジェクト）を返す。`find_previous_siblings()` メソッドはすべての前の兄弟要素のリスト `bs4.ResultSet` オブジェクトを返す。引数の意味は、`find()` および `find_all()` と同様である。

``` python
find_next_sibling(name=None, attrs={}, string=None, **kwargs)
find_next_siblings(name=None, attrs={}, string=None, limit=None, **kwargs)
```

このメソッドは、引数に指定した検索条件にマッチした次の兄弟要素（タグオブジェクトか bs4.NavigableString オブジェクト）を返す。`find_next_siblings` メソッドはすべての次の兄弟要素のリスト `bs4.ResultSet` オブジェクトを返す。引数の意味は、`find()` および `find_all()` と同様である。

ツリーの変更
------------

``` python
soup.new_tag(name, namespace=None, nsprefix=None, attrs={}, sourceline=None, sourcepos=None, **kwattrs)
```

これは、`bs4.BeautifulSoup` オブジェクトのメソッドで、新しいタグを作成し、タグオブジェクトを返す。主な引数は次のとおり。

| 引数 | 意味 |
|:---|:---|
| `name` | 新しいタグの名前 |
| `attrs` | 新しいタグの属性の名前と値を辞書で指定する |
| `kwattrs` | 新しいタグの属性の名前と値をキーワード引数で指定する |

``` python
soup.new_string(s, subclass=None):
```

これは、`bs4.BeautifulSoup` オブジェクトのメソッドで、新しいテキストを作成し、`bs4.NavigableString` オブジェクトを返す。`subclass` に `bs4.NavigableString` のサブクラスを指定できる。

``` python
soup.wrap(wrap_inside)
```

このメソッドは、このオブジェクトの親要素として `wrap_inside` に指定したタグオブジェクトをツリーに挿入する。

``` python
soup.unwrap()
```

このメソッドは、このオブジェクトだけを取り除いて、子要素は残す。

``` python
soup.extract()
```

このメソッドは、このオブジェクトを削除し、親要素を返す。

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup("<p></p>")
new_div = soup.new_tag("div")
print(soup.p.wrap(new_div))
print(soup.p.unwrap())

<div><p></p></div>
<p></p>
